In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%qtconsole

In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.INFO)

from src.parameters import ANIMALS, SAMPLING_FREQUENCY

In [3]:
from loren_frank_data_processing import make_epochs_dataframe

epoch_info = make_epochs_dataframe(ANIMALS)

# DIO

In [4]:
from loren_frank_data_processing.DIO import get_DIO

epoch_key = ('dav', 2, 2)
dio = get_DIO(epoch_key, ANIMALS)

ValueError: no field of name times

In [61]:
import numpy as np
import pandas as pd

from loren_frank_data_processing.core import get_data_structure
from loren_frank_data_processing.tetrodes import get_trial_time

_PULSE_TIMES_NAMES = ['times', 'pulsetimes']
_PULSE_IND_NAMES = ['values', 'pulse_ind']


def get_DIO(epoch_key, animals):
    ''''''
    animal, day, epoch = epoch_key
    pins = get_data_structure(
        animals[animal], day, 'DIO', 'DIO')[epoch - 1].squeeze()
    pins_df = []

    for pin_ind, pin in enumerate(pins):
        if pin.dtype.names is not None:
            pulse_times_ind = np.nonzero(
                [name in _PULSE_TIMES_NAMES
                 for name in pin.dtype.names])[0].item()
            pulse_times_field_name = pin.dtype.names[pulse_times_ind]
            try:
                times = pin[pulse_times_field_name][0, 0].squeeze()
                time = pd.to_timedelta(times.flat, unit='us')
            except ValueError:
                times = pin[pulse_times_field_name][0, 0].item()
                time = pd.to_timedelta(times, unit='us')

            try:
                values = pin['values'][0, 0].squeeze()
            except ValueError:
                values = np.zeros_like(times)
                values[:, 1] = 1
                values = values.flat
            try:
                pin_id = pin['original_id'][0, 0].item()
            except ValueError:
                pin_id = pin_ind

            try:
                series = pd.Series(values, index=time, name=pin_id)
            except TypeError:
                series = pd.Series(name=pin_id)

            pins_df.append(series.loc[series.index.drop_duplicates(keep=False)])
    return pd.concat(pins_df, axis=1).fillna(0).sort_index()

In [62]:
epoch_key = ('dav', 2, 2)
dio = get_DIO(epoch_key, ANIMALS)

In [63]:
def get_DIO_indicator(epoch_key, animals, time_function=get_trial_time):
    time = time_function(epoch_key, animals)
    dio = get_DIO(epoch_key, animals)
    time_index = np.digitize(dio.index.total_seconds(),
                             time.total_seconds())
    time_index[time_index == time.size] = time.size - 1
    return (dio.groupby(time[time_index]).sum()
            .reindex(index=time, fill_value=0))

In [64]:
from loren_frank_data_processing import get_position_dataframe

get_DIO_indicator(epoch_key, ANIMALS)

,6,7,22,23
time,,,,
01:40:29.980800,33.0,35.0,55.0,58.0
01:40:29.981466,0.0,0.0,0.0,0.0
01:40:29.982133,0.0,0.0,0.0,0.0
01:40:29.982800,0.0,0.0,0.0,0.0
01:40:29.983466,0.0,0.0,0.0,0.0
01:40:29.984133,0.0,0.0,0.0,0.0
01:40:29.984800,0.0,0.0,0.0,0.0
01:40:29.985466,0.0,0.0,0.0,0.0
01:40:29.986133,0.0,0.0,0.0,0.0


## Spike Indicators

In [66]:
from loren_frank_data_processing import get_all_spike_indicators, make_neuron_dataframe

epoch_key = ('dud', 5, 2)

neuron_info = make_neuron_dataframe(ANIMALS).xs(epoch_key, drop_level=False)

In [68]:
brain_areas = ['CA1', 'CA3']

neuron_info = neuron_info.loc[
    (neuron_info.numspikes > 0) &
    neuron_info.area.isin(brain_areas)]
spikes = get_all_spike_indicators(neuron_info.index, ANIMALS)


IndexError: index 1767424 is out of bounds for axis 1 with size 1767424

In [101]:
from loren_frank_data_processing.neurons import get_spikes_dataframe, get_trial_time

def get_all_spike_indicators(neuron_keys, animals,
                             time_function=get_trial_time):
    time = time_function(neuron_keys[0][:3], animals)
    spikes_dfs = [get_spikes_dataframe(neuron_key, animals)
                  for neuron_key in neuron_keys]
    n_time = time.size
    for ind, spikes_df in enumerate(spikes_dfs):
        spike_time_ind = np.digitize(
            spikes_df.index.total_seconds(), time.total_seconds())
        spike_time_ind = spike_time_ind[spike_time_ind < n_time]
        spikes_dfs[ind] = (spikes_df
                           .iloc[spike_time_ind < n_time]
                           .groupby(time[spike_time_ind]).sum()
                           .reindex(index=time, fill_value=0))

    return pd.concat(spikes_dfs, axis=1)

In [102]:
spikes = get_all_spike_indicators(neuron_info.index, ANIMALS)